Modified from http://www.johnmyleswhite.com/notebook/2013/01/07/symbolic-differentiation-in-julia/

In [36]:
expr = :(x + 1)

:(x + 1)

In [37]:
typeof(expr)

Expr

In [38]:
expr |> dump

Expr 
  head: Symbol call
  args: Array(Any,(3,))
    1: Symbol +
    2: Symbol x
    3: Int64 1
  typ: Any


In [39]:
@doc Expr

No documentation found.

**Summary:**

```julia
type Expr <: Any
```

**Fields:**

```julia
head :: Symbol
args :: Array{Any,1}
typ  :: Any
```


In [40]:
expr2 = Expr(:call, :+, 1, 1)

:(1 + 1)

In [41]:
expr2 |> eval

2

In [42]:
# Constant
differentiate(x::Number, target::Symbol) = 0

differentiate (generic function with 3 methods)

In [43]:
differentiate(5, :x)

0

In [44]:
# Just one symbol
function differentiate(s::Symbol, target::Symbol)
    if s == target
        return 1
    else
        return 0
    end
end

differentiate (generic function with 3 methods)

In [45]:
differentiate(:x, :x)

1

In [46]:
differentiate(:y, :x)

0

In [47]:
# Sum
function differentiate_sum(ex::Expr, target::Symbol)
    #@printf "diff[+]: %s\n" ex
    n = length(ex.args)
    new_args = Array(Any, n)
    new_args[1] = :+
    for i in 2:n
        # @printf "diff: %s\n" ex.args[i]
        new_args[i] = differentiate(ex.args[i], target)
    end
    return Expr(:call, new_args...)
end

differentiate_sum (generic function with 1 method)

In [48]:
differentiate_sum(:(x + y), :x)

:(1 + 0)

In [49]:
function differentiate_subtraction(ex::Expr, target::Symbol)
    #@printf "diff[-]: %s\n" ex
    n = length(ex.args)
    new_args = Array(Any, n)
    new_args[1] = :-
    for i in 2:n
        new_args[i] = differentiate(ex.args[i], target)
    end
    return Expr(:call, new_args...)
end

differentiate_subtraction (generic function with 1 method)

In [50]:
differentiate_subtraction(:(x - y), :x)

:(1 - 0)

In [51]:
function differentiate_product(ex::Expr, target::Symbol)
    #@printf "diff[*]: %s\n" ex
    n = length(ex.args)
    res_args = Array(Any, n)
    res_args[1] = :+
    for i in 2:n
       new_args = Array(Any, n)
       new_args[1] = :*
       for j in 2:n
           if j == i
               new_args[j] = differentiate(ex.args[j], target)
           else
               new_args[j] = ex.args[j]
           end
       end
       res_args[i] = Expr(:call, new_args...)
    end
    return Expr(:call, res_args...)
end

differentiate_product (generic function with 1 method)

In [52]:
differentiate_product(:(x * x), :x)

:(1x + x * 1)

In [53]:
function differentiate_quotient(ex::Expr, target::Symbol)
    #@printf "diff[/]: %s\n" ex
    return Expr(:call,
                :/,
                Expr(:call,
                    :-,
                    Expr(:call,
                        :*,
                        differentiate(ex.args[2], target),
                        ex.args[3]
                    ),
                    Expr(:call,
                        :*,
                        ex.args[2],
                        differentiate(ex.args[3], target)
                    )
                ),
                Expr(:call,
                    :^,
                    ex.args[3],
                    2
                )
            )
end

differentiate_quotient (generic function with 1 method)

In [54]:
differentiate_quotient(:(x / y), :y)

:((0y - x * 1) / y ^ 2)

In [55]:
differentiate_lookup = Dict(
  :+ => differentiate_sum,
  :- => differentiate_subtraction,
  :* => differentiate_product,
  :/ => differentiate_quotient
)

Dict{Symbol,Function} with 4 entries:
  :/ => differentiate_quotient
  :+ => differentiate_sum
  :* => differentiate_product
  :- => differentiate_subtraction

In [56]:
function differentiate(ex::Expr, target::Symbol)
    #@printf "diff: %s\n" ex
    if ex.head == :call
        if haskey(differentiate_lookup, ex.args[1])
            return differentiate_lookup[ex.args[1]](ex, target)
        else
            error("Don't know how to differentiate $(ex.args[1])")
        end
    else
        error("ex.head != :call")
        # return differentiate(ex.head)
    end
end

differentiate (generic function with 3 methods)

In [57]:
differentiate(:(x * x), :x)

:(1x + x * 1)

In [58]:
differentiate(:((x + x) * x), :x)

:((1 + 1) * x + (x + x) * 1)

In [59]:
differentiate(:(1 + x + x * x), :x)

:(0 + 1 + (1x + x * 1))

In [60]:
differentiate(:(1 + z + x * z + x * z * z), :z)

:(0 + 1 + (0z + x * 1) + (0 * z * z + x * 1 * z + x * z * 1))

In [61]:
differentiate(:(1 + x + 1 / x + x * x), :x)

:(0 + 1 + (0x - 1 * 1) / x ^ 2 + (1x + x * 1))

In [62]:
Pkg.add("Calculus")

INFO: Nothing to be done


In [63]:
import Calculus